**Atividade módulo 4 - Intro visão computacional**

Minha intenção é que vocês juntem diferentes partes do que foi apresentado aqui para criar um possível processo, inspirado na R-CNN, de identificação de objetos em imagens.

Primeiramente, peço que revejam o selective search, para ver se o processo ficou claro, e se ficou claro como pegar as sub-imagens que ele seleciona para cada imagem. Assim que estiverem ok com essa parte, a principal parte da tarefa é fazer um pipeline que pega uma imagem de input, usa o selective search nela, e usa essas saídas do selective search como entrada em uma rede neural classificadora. Dessa maneira, para uma imagem de input vamos ter diversas sugestões de bounding boxes (propostas pelo selective search) e para cada uma delas, teremos uma probabiblidade associada daquela bounding boxe conter um objeto.

Podemos usar a VGG-19 que usamos na última aula como rede classificadora (lembrem: se usarmos ela treinada com a imageNet, ela vai conseguir classificar dentro de 1000 classes) e fazer um bom classificador sem gastar muita energia em treinamento.

Mas o que eu propus, para facilitar essa classificação, era usar a seguinte base de dados: https://www.kaggle.com/c/dogs-vs-cats que só tem duas classes. Assim vai ficar mais fácil de avaliar se o que a rede está fazendo faz sentido. 

In [1]:
import os
import cv2
import numpy as np
from keras.applications.vgg19 import VGG19, preprocess_input

In [2]:
# Criando instância da rede VGG-19 pré-treinada
model = VGG19(weights='imagenet', include_top=True)

In [3]:
# Definindo função que extrai recursos de cada imagem
def extract_features(img_path):
    # Carregando e redimensionando a imagem para tamanho esperado pelo modelo
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    # Pre-processando imagem para ser usada como entrada do modelo
    img = preprocess_input(img)
    # Obtendo previsões da rede
    features = model.predict(np.array([img]))
    # Retornando características extraídas da imagem
    return features.flatten()

In [4]:
# Pasta com as imagens de treinamento -> data_dir
data_folder = 'dogs-vs-cats/train'

In [5]:
# Listando arquivos de imagem na pasta
img_files = os.listdir(data_folder)

In [6]:
# Iterando em cada imagem
features_list = []
for img_file in img_files:
    img_path = os.path.join(data_folder, img_file)
    features = extract_features(img_path)
    features_list.append(features)

1/1 [==============================] - 2s 2s/step


In [7]:
# Convertendo lista de features em array
X = np.array(features_list)

In [8]:
# Rotulando imagens (0 cães - 1 gatos)
y = np.array([0 if 'dog' in f else 1 for f in img_files])

In [9]:
# Treinando modelo de classificação com features listadas
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X, y)

LinearSVC()

In [10]:
# Pasta com as imagens de teste
test_folder = 'dogs-vs-cats/test1'

In [11]:
# Listando arquivos de imagem na pasta
test_files = os.listdir(test_folder)

In [14]:
# Iterando sobre cada imagem, extraindo recursos e fazendo previsões
for test_file in test_files:
    test_path = os.path.join(test_folder, test_file)
    features = extract_features(test_path)
    prediction = clf.predict(features.reshape(1, -1))[0]
    if prediction == 0:
        print(test_file, "é um cão")
    else:
        print(test_file, "é um gato")

1/1 [==============================] - 5s 5s/step
1.jpg é um cão
1/1 [==============================] - 5s 5s/step
10.jpg é um gato
1/1 [==============================] - 5s 5s/step
100.jpg é um gato
1/1 [==============================] - 5s 5s/step
1000.jpg é um cão
1/1 [==============================] - 5s 5s/step
10000.jpg é um cão
1/1 [==============================] - 5s 5s/step
10001.jpg é um gato
1/1 [==============================] - 5s 5s/step
10002.jpg é um gato
1/1 [==============================] - 6s 6s/step
10003.jpg é um cão
1/1 [==============================] - 7s 7s/step
10004.jpg é um cão
1/1 [==============================] - 5s 5s/step
10005.jpg é um gato
1/1 [==============================] - 5s 5s/step
10006.jpg é um gato
1/1 [==============================] - 5s 5s/step
10007.jpg é um gato
1/1 [==============================] - 5s 5s/step
10008.jpg é um gato
1/1 [==============================] - 5s 5s/step
10009.jpg é um gato
1/1 [==============================

In [16]:
# Conclusão

# Este modelo de classificação de imagens, desenvolvido com base na rede neural VGG-19 e no SVM teve uma precisão ao 
# detectar cães e gatos nas imagens.